# Considerações

Este notebook tem como objetivo explorar a base de dados das conversas da Alzira.

Ao longo da análise, realizaremos uma exploração inicial dos dados, faremos o processamento da base e utilizaremos a API da OpenAI para desenvolver o Gregório, a IA responsável por analisar as conversas. Por fim, examinaremos os resultados obtidos.

A partir deste notebook, extrairei apenas as funções essenciais para a implementação do framework completo em um arquivo .py.

# Bibliotecas

In [58]:
import psycopg2
import os

import numpy as np
import pandas as pd

import openai
import json

from dotenv import load_dotenv

# Conectando ao banco de dados

In [63]:
#load_dotenv(dotenv_path="../.env")
load_dotenv()

True

In [64]:
DB_HOST = "localhost"
DB_PORT = "5432"
DB_USER = os.getenv("POSTGRES_PASSWORD")
DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB_NAME = os.getenv("POSTGRES_DB")

conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    dbname=DB_NAME
)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


# Extração das conversas de modo bruto

In [4]:
cur = conn.cursor()
cur.execute("SELECT id, motel_id, session_id, content, remote, created_at FROM message")
rows = cur.fetchall()

In [5]:
print(type(rows))
print(type(rows[0]))

<class 'list'>
<class 'tuple'>


In [6]:
df_messages = pd.DataFrame(rows, columns = ['message_id', 'motel_id', 'session_id', 'content', 'remote', 'created_at'])
df_messages

,message_id,motel_id,session_id,content,remote,created_at
0,41,3,3,Olá,True,2024-11-27 21:27:48
1,42,3,3,"Olá, Roberto! 😊 Eu sou a Alzira, do Intense Mo...",False,2024-11-27 21:28:07
2,43,3,3,Valor,True,2024-11-27 21:28:19
3,44,3,3,"Claro! 😊 Para verificar os valores, você poder...",False,2024-11-27 21:28:37
4,45,3,3,Hoje a noite,True,2024-11-27 21:28:49
...,...,...,...,...,...,...
236,1493,3,18,nenhuma,True,2025-01-29 18:35:16
237,1494,3,18,Sem problemas! 😊\n\nVamos confirmar suas escol...,False,2025-01-29 18:35:37
238,1495,3,18,A sua reserva está quase pronta! 🎉\n\nAqui est...,False,2025-01-29 18:35:53
239,1496,3,18,"somente isso , obrigada",True,2025-01-29 18:36:47


In [7]:
df = df_messages.copy()

df = df.sort_values(by=['session_id','created_at']).reset_index(drop = True)

# Análise simples

## Janela de tempo em que ocorreu as conversas

In [8]:
(df['created_at'].min(), df['created_at'].max())

(Timestamp('2024-11-27 21:27:48'), Timestamp('2025-01-29 18:37:06'))

In [9]:
df.groupby(['session_id'])['created_at'].agg(lambda x: x.max() - x.min())

session_id
3    40 days 22:17:40
5     0 days 00:19:02
6     1 days 02:47:01
11    0 days 00:18:28
13    0 days 00:38:15
14    0 days 00:22:09
15    0 days 00:29:07
16    0 days 00:48:28
17    0 days 00:19:52
18    0 days 00:08:28
Name: created_at, dtype: timedelta64[ns]

In [10]:
df_aux = df.groupby(['session_id'])['created_at'].agg(['min', 'max'])
df_aux['intervalo_tempo'] = df_aux['max'] - df_aux['min']
df_aux

,min,max,intervalo_tempo
session_id,,,
3,2024-11-27 21:27:48,2025-01-07 19:45:28,40 days 22:17:40
5,2025-01-07 12:10:47,2025-01-07 12:29:49,0 days 00:19:02
6,2025-01-07 12:28:02,2025-01-08 15:15:03,1 days 02:47:01
11,2025-01-08 17:46:43,2025-01-08 18:05:11,0 days 00:18:28
13,2025-01-12 10:51:02,2025-01-12 11:29:17,0 days 00:38:15
14,2025-01-17 00:20:38,2025-01-17 00:42:47,0 days 00:22:09
15,2025-01-22 16:42:16,2025-01-22 17:11:23,0 days 00:29:07
16,2025-01-27 14:12:34,2025-01-27 15:01:02,0 days 00:48:28
17,2025-01-28 14:55:35,2025-01-28 15:15:27,0 days 00:19:52


In [11]:
df.loc[df['session_id']==3]

,message_id,motel_id,session_id,content,remote,created_at
0,41,3,3,Olá,True,2024-11-27 21:27:48
1,42,3,3,"Olá, Roberto! 😊 Eu sou a Alzira, do Intense Mo...",False,2024-11-27 21:28:07
2,43,3,3,Valor,True,2024-11-27 21:28:19
3,44,3,3,"Claro! 😊 Para verificar os valores, você poder...",False,2024-11-27 21:28:37
4,45,3,3,Hoje a noite,True,2024-11-27 21:28:49
5,46,3,3,Perfeito! Que horas você gostaria de se hosped...,False,2024-11-27 21:29:07
6,47,3,3,20h,True,2024-11-27 21:29:42
7,48,3,3,Aqui estão as suítes disponíveis para hoje à n...,False,2024-11-27 21:30:02
8,49,3,3,<PHOTO>\n\n*suíte imaginarium*\n_(garagem priv...,False,2024-11-27 21:30:04
9,51,3,3,"<PHOTO>\n\n*suíte ilusion*\n_(hidro, garagem p...",False,2024-11-27 21:30:07


In [12]:
df['diff_tempo_entre_mensagens'] = df.groupby(['session_id'])['created_at'].diff()

In [13]:
df.sort_values(by = ['diff_tempo_entre_mensagens'], ascending = False).head(10)

,message_id,motel_id,session_id,content,remote,created_at,diff_tempo_entre_mensagens
41,305,3,3,Reserva,True,2025-01-07 19:19:35,31 days 17:37:56
22,104,3,3,Oi,True,2024-12-07 01:22:01,9 days 00:50:11
92,565,3,6,Bom dia,True,2025-01-08 15:04:59,1 days 01:58:15
18,60,3,3,Quais opções do cardápio?,True,2024-11-28 00:10:13,0 days 01:20:19
13,55,3,3,Ilusion,True,2024-11-27 22:48:43,0 days 01:01:09
20,62,3,3,Tem promoção?,True,2024-11-28 00:31:23,0 days 00:20:49
132,899,3,13,Posso continuar o atendimento? 😊,False,2025-01-12 11:29:17,0 days 00:19:51
178,1393,3,16,Posso te ajudar com mais alguma coisa? 😊,False,2025-01-27 14:34:02,0 days 00:19:00
45,268,3,3,¿Puedo continuar ayudándote con tu reserva? 😊,False,2025-01-07 19:45:28,0 days 00:18:38
139,1240,3,14,Posso continuar te ajudando com mais alguma in...,False,2025-01-17 00:41:33,0 days 00:18:21


In [14]:
df.loc[df['session_id']==3]

,message_id,motel_id,session_id,content,remote,created_at,diff_tempo_entre_mensagens
0,41,3,3,Olá,True,2024-11-27 21:27:48,NaT
1,42,3,3,"Olá, Roberto! 😊 Eu sou a Alzira, do Intense Mo...",False,2024-11-27 21:28:07,0 days 00:00:19
2,43,3,3,Valor,True,2024-11-27 21:28:19,0 days 00:00:12
3,44,3,3,"Claro! 😊 Para verificar os valores, você poder...",False,2024-11-27 21:28:37,0 days 00:00:18
4,45,3,3,Hoje a noite,True,2024-11-27 21:28:49,0 days 00:00:12
5,46,3,3,Perfeito! Que horas você gostaria de se hosped...,False,2024-11-27 21:29:07,0 days 00:00:18
6,47,3,3,20h,True,2024-11-27 21:29:42,0 days 00:00:35
7,48,3,3,Aqui estão as suítes disponíveis para hoje à n...,False,2024-11-27 21:30:02,0 days 00:00:20
8,49,3,3,<PHOTO>\n\n*suíte imaginarium*\n_(garagem priv...,False,2024-11-27 21:30:04,0 days 00:00:02
9,51,3,3,"<PHOTO>\n\n*suíte ilusion*\n_(hidro, garagem p...",False,2024-11-27 21:30:07,0 days 00:00:03


In [15]:
df.loc[df['diff_tempo_entre_mensagens'] >= pd.Timedelta("12 hours") ]

,message_id,motel_id,session_id,content,remote,created_at,diff_tempo_entre_mensagens
22,104,3,3,Oi,True,2024-12-07 01:22:01,9 days 00:50:11
41,305,3,3,Reserva,True,2025-01-07 19:19:35,31 days 17:37:56
92,565,3,6,Bom dia,True,2025-01-08 15:04:59,1 days 01:58:15


In [16]:
df['flag'] = 0

df.loc[df['diff_tempo_entre_mensagens'] >= pd.Timedelta("12 hours"), 'flag'] = 1

df['cum_flag'] = df.groupby('session_id')['flag'].cumsum()

In [17]:
aux_unique = df.groupby('session_id')['cum_flag'].unique().sort_index()
aux_unique

session_id
3     [0, 1, 2]
5           [0]
6        [0, 1]
11          [0]
13          [0]
14          [0]
15          [0]
16          [0]
17          [0]
18          [0]
Name: cum_flag, dtype: object

In [18]:
aux_nunique = df.groupby('session_id')['cum_flag'].nunique().sort_index()
aux_nunique

session_id
3     3
5     1
6     2
11    1
13    1
14    1
15    1
16    1
17    1
18    1
Name: cum_flag, dtype: int64

# Observação

Há algumas conversas diferentes que aparentemente possuem o mesmo ID, vou fazer uma separação delas no momento em que o tempo entre uma mensagem e outra é maior que 12 horas. 

In [19]:
np.random.seed(10)

for session_id in aux_nunique[aux_nunique>1].index.tolist():

    for cum_flag in aux_unique[aux_unique.index==session_id].values[0].tolist():
        
        new_id = np.random.randint(1, 30)
        ids_existentes = df['session_id'].unique().tolist()
        
        while new_id in ids_existentes:
            new_id = np.random.randint(1, 30)

        df.loc[(df['session_id']==session_id) & (df['cum_flag'] == cum_flag), 'session_id'] = new_id

In [20]:
df['min_created_session'] = df.groupby(['session_id'])['created_at'].transform("min")
df

,message_id,motel_id,session_id,content,remote,created_at,diff_tempo_entre_mensagens,flag,cum_flag,min_created_session
0,41,3,10,Olá,True,2024-11-27 21:27:48,NaT,0,0,2024-11-27 21:27:48
1,42,3,10,"Olá, Roberto! 😊 Eu sou a Alzira, do Intense Mo...",False,2024-11-27 21:28:07,0 days 00:00:19,0,0,2024-11-27 21:27:48
2,43,3,10,Valor,True,2024-11-27 21:28:19,0 days 00:00:12,0,0,2024-11-27 21:27:48
3,44,3,10,"Claro! 😊 Para verificar os valores, você poder...",False,2024-11-27 21:28:37,0 days 00:00:18,0,0,2024-11-27 21:27:48
4,45,3,10,Hoje a noite,True,2024-11-27 21:28:49,0 days 00:00:12,0,0,2024-11-27 21:27:48
...,...,...,...,...,...,...,...,...,...,...
236,1493,3,18,nenhuma,True,2025-01-29 18:35:16,0 days 00:01:43,0,0,2025-01-29 18:28:38
237,1494,3,18,Sem problemas! 😊\n\nVamos confirmar suas escol...,False,2025-01-29 18:35:37,0 days 00:00:21,0,0,2025-01-29 18:28:38
238,1495,3,18,A sua reserva está quase pronta! 🎉\n\nAqui est...,False,2025-01-29 18:35:53,0 days 00:00:16,0,0,2025-01-29 18:28:38
239,1496,3,18,"somente isso , obrigada",True,2025-01-29 18:36:47,0 days 00:00:54,0,0,2025-01-29 18:28:38


In [21]:
df = df.sort_values(by = ['min_created_session', 'created_at']).reset_index(drop = True)
df

,message_id,motel_id,session_id,content,remote,created_at,diff_tempo_entre_mensagens,flag,cum_flag,min_created_session
0,41,3,10,Olá,True,2024-11-27 21:27:48,NaT,0,0,2024-11-27 21:27:48
1,42,3,10,"Olá, Roberto! 😊 Eu sou a Alzira, do Intense Mo...",False,2024-11-27 21:28:07,0 days 00:00:19,0,0,2024-11-27 21:27:48
2,43,3,10,Valor,True,2024-11-27 21:28:19,0 days 00:00:12,0,0,2024-11-27 21:27:48
3,44,3,10,"Claro! 😊 Para verificar os valores, você poder...",False,2024-11-27 21:28:37,0 days 00:00:18,0,0,2024-11-27 21:27:48
4,45,3,10,Hoje a noite,True,2024-11-27 21:28:49,0 days 00:00:12,0,0,2024-11-27 21:27:48
...,...,...,...,...,...,...,...,...,...,...
236,1493,3,18,nenhuma,True,2025-01-29 18:35:16,0 days 00:01:43,0,0,2025-01-29 18:28:38
237,1494,3,18,Sem problemas! 😊\n\nVamos confirmar suas escol...,False,2025-01-29 18:35:37,0 days 00:00:21,0,0,2025-01-29 18:28:38
238,1495,3,18,A sua reserva está quase pronta! 🎉\n\nAqui est...,False,2025-01-29 18:35:53,0 days 00:00:16,0,0,2025-01-29 18:28:38
239,1496,3,18,"somente isso , obrigada",True,2025-01-29 18:36:47,0 days 00:00:54,0,0,2025-01-29 18:28:38


## Tempo de duração das conversas

In [22]:
df.groupby('session_id')['created_at'].agg(lambda x: x.max() - x.min())

session_id
1    0 days 00:19:38
5    0 days 00:19:02
10   0 days 03:04:02
11   0 days 00:18:28
13   0 days 00:38:15
14   0 days 00:22:09
15   0 days 00:29:07
16   0 days 00:48:28
17   0 days 00:19:52
18   0 days 00:08:28
26   0 days 00:10:04
28   0 days 00:25:53
29   0 days 00:38:42
Name: created_at, dtype: timedelta64[ns]

## Quantidade de conversas e média de mensagens por conversa

In [23]:
df['session_id'].nunique()

13

In [24]:
df.shape[0]/df['session_id'].nunique()

18.53846153846154

In [25]:
df.groupby('session_id')['message_id'].nunique().sort_values()

session_id
28     5
14     9
13    10
17    13
26    13
11    18
1     19
15    22
10    22
5     23
29    23
18    26
16    38
Name: message_id, dtype: int64

## Móteis

In [26]:
df['motel_id'].value_counts()

motel_id
3    241
Name: count, dtype: int64

## Tempo de resposta da Alzira

In [27]:
df['prev_remote'] = df.groupby(['session_id'])['remote'].shift()
df['prev_created_at'] = df.groupby(['session_id'])['created_at'].shift()

In [28]:
df['prev_remote'] = df.groupby(['session_id'])['remote'].shift()
df['prev_created_at'] = df.groupby(['session_id'])['created_at'].shift()

mask = ((df['remote'] == False) & (df['prev_remote'] == True))

df['tempo_resposta'] = np.nan

df.loc[mask, 'tempo_resposta'] = df['created_at'] - df['prev_created_at']

C:\Users\natha\AppData\Local\Temp\ipykernel_18344\106213065.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<TimedeltaArray>
['0 days 00:00:19', '0 days 00:00:18', '0 days 00:00:18', '0 days 00:00:20',
 '0 days 00:00:20', '0 days 00:00:24', '0 days 00:00:21', '0 days 00:00:27',
 '0 days 00:00:19', '0 days 00:00:18', '0 days 00:00:17', '0 days 00:00:22',
 '0 days 00:00:19', '0 days 00:00:25', '0 days 00:00:19', '0 days 00:00:18',
 '0 days 00:00:18', '0 days 00:00:23', '0 days 00:00:20', '0 days 00:00:26',
 '0 days 00:00:29', '0 days 00:00:20', '0 days 00:00:18', '0 days 00:00:20',
 '0 days 00:00:19', '0 days 00:00:21', '0 days 00:00:17', '0 days 00:00:29',
 '0 days 00:00:20', '0 days 00:00:18', '0 days 00:00:21', '0 days 00:00:18',
 '0 days 00:00:24', '0 days 00:00:18', '0 days 00:00:29', '0 days 00:00:19',
 '0 days 00:00:21', '0 days 00:00:22', '0 days 00:00:19', '0 days 00:00:20',
 '0 days 00:00:18'

In [29]:
df

,message_id,motel_id,session_id,content,remote,created_at,diff_tempo_entre_mensagens,flag,cum_flag,min_created_session,prev_remote,prev_created_at,tempo_resposta
0,41,3,10,Olá,True,2024-11-27 21:27:48,NaT,0,0,2024-11-27 21:27:48,NaN,NaT,NaN
1,42,3,10,"Olá, Roberto! 😊 Eu sou a Alzira, do Intense Mo...",False,2024-11-27 21:28:07,0 days 00:00:19,0,0,2024-11-27 21:27:48,True,2024-11-27 21:27:48,0 days 00:00:19
2,43,3,10,Valor,True,2024-11-27 21:28:19,0 days 00:00:12,0,0,2024-11-27 21:27:48,False,2024-11-27 21:28:07,NaN
3,44,3,10,"Claro! 😊 Para verificar os valores, você poder...",False,2024-11-27 21:28:37,0 days 00:00:18,0,0,2024-11-27 21:27:48,True,2024-11-27 21:28:19,0 days 00:00:18
4,45,3,10,Hoje a noite,True,2024-11-27 21:28:49,0 days 00:00:12,0,0,2024-11-27 21:27:48,False,2024-11-27 21:28:37,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,1493,3,18,nenhuma,True,2025-01-29 18:35:16,0 days 00:01:43,0,0,2025-01-29 18:28:38,False,2025-01-29 18:33:33,NaN
237,1494,3,18,Sem problemas! 😊\n\nVamos confirmar suas escol...,False,2025-01-29 18:35:37,0 days 00:00:21,0,0,2025-01-29 18:28:38,True,2025-01-29 18:35:16,0 days 00:00:21
238,1495,3,18,A sua reserva está quase pronta! 🎉\n\nAqui est...,False,2025-01-29 18:35:53,0 days 00:00:16,0,0,2025-01-29 18:28:38,False,2025-01-29 18:35:37,NaN
239,1496,3,18,"somente isso , obrigada",True,2025-01-29 18:36:47,0 days 00:00:54,0,0,2025-01-29 18:28:38,False,2025-01-29 18:35:53,NaN


In [30]:
df.groupby('session_id')['tempo_resposta'].mean()

session_id
1               0 days 00:00:20
5     0 days 00:00:21.857142857
10       0 days 00:00:20.875000
11       0 days 00:00:20.250000
13    0 days 00:00:19.333333333
14              0 days 00:00:20
15              0 days 00:00:16
16    0 days 00:00:22.111111111
17    0 days 00:00:19.666666666
18    0 days 00:00:21.833333333
26              0 days 00:00:22
28              0 days 00:00:19
29    0 days 00:00:20.571428571
Name: tempo_resposta, dtype: object

# Configuração da API OpenAI

In [31]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()


# Processamento e análise das conversas

## Prompt

In [32]:
def analisar_conversa(texto_conversa, temperature = 0.0):
    """
    Envia para o ChatGPT (modelo) instruções fixas + texto da conversa.
    Espera receber JSON com "satisfaction", "summary" e "improvement".
    """

    system_prompt = (
        "Você é um analista de conversas."
        "São conversas entre cliente (sinalizada por 'user') e um chatbot (sinalizado por 'Alzira') para a reserva de quarto de motel"
        "As conversas incluem o dia e hora que foi enviada cada mensagem, o tempo que demorou para o chatbot responder a última mensagem do cliente"
        "As mensagens estão separadas por algumas linhas em branco e diversos hífens"
        "Preciso que, para cada conversa que eu fornecer, você: "
        "1) Dê uma nota de 0 a 10 para a satisfação do cliente, levando em consideração principalmente a forma que o cliente reagiu às mensagens mas também o tempo de resposta. "
        "2) Faça um breve resumo da conversa. "
        "3) Forneça como a conversa poderia ter sido melhor. "
        "4) Indica se houve de fato a reserva ou não. True se houve e False se não"
        "5) Forneça o valor que foi faturado com o cliente, em R$, levando em consideração o valor do quarto que o cliente reservou e adicionais. Se não houve reserva, o valor é 0"
        "6) Data e hora de quando ficou agendada a reserva. Se não houve reserva, retorna nulo. Essa data é a da reserva e não quando começou ou terminou a conversa"
        "Responda APENAS em JSON no formato: "
        '{"satisfaction": <numero>, "summary": "<texto>", "improvement": "<texto>", "reserva":"<bool>", "income": "<numero>", "date": "<datetime>"}'
    )

    user_prompt = (
        f"Aqui está a conversa:\n\n{texto_conversa}\n\n"
        "Lembre: Retorne exclusivamente JSON e nada mais! Sem códigos de linguagem natural, sem blocos de código, sem texto adicional. Somente o objeto JSON."
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # ou outro modelo que você use
        messages=[
            {"role": "developer", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )

    resposta = response.choices[0].message.content
    
    return resposta

In [33]:
df.head()

,message_id,motel_id,session_id,content,remote,created_at,diff_tempo_entre_mensagens,flag,cum_flag,min_created_session,prev_remote,prev_created_at,tempo_resposta
0,41,3,10,Olá,True,2024-11-27 21:27:48,NaT,0,0,2024-11-27 21:27:48,NaN,NaT,NaN
1,42,3,10,"Olá, Roberto! 😊 Eu sou a Alzira, do Intense Mo...",False,2024-11-27 21:28:07,0 days 00:00:19,0,0,2024-11-27 21:27:48,True,2024-11-27 21:27:48,0 days 00:00:19
2,43,3,10,Valor,True,2024-11-27 21:28:19,0 days 00:00:12,0,0,2024-11-27 21:27:48,False,2024-11-27 21:28:07,NaN
3,44,3,10,"Claro! 😊 Para verificar os valores, você poder...",False,2024-11-27 21:28:37,0 days 00:00:18,0,0,2024-11-27 21:27:48,True,2024-11-27 21:28:19,0 days 00:00:18
4,45,3,10,Hoje a noite,True,2024-11-27 21:28:49,0 days 00:00:12,0,0,2024-11-27 21:27:48,False,2024-11-27 21:28:37,NaN


In [34]:
df_conversas = pd.DataFrame()

for session_id in df.session_id.unique().tolist():

    df_aux = df.loc[df['session_id']==session_id].sort_values(by='created_at')

    string = ""

    for i, row in df_aux.iterrows():

        if row['remote'] == True:
            individuo = "User"
            string_aux = f"Dia e hora da mensagem: {row['created_at']} \n {individuo}: {row['content']} \n\n {100 * '-'} \n"
        
        elif row['remote'] == False:
            tempo_resposta = row['tempo_resposta']
            individuo = "Alzira"
            string_aux = f"Dia e hora da mensagem: {row['created_at']} \n Tempo de resposta: {tempo_resposta} \n {individuo}: {row['content']}  \n\n {100 * '-'} \n"

        string = string + string_aux

    resposta = analisar_conversa(string)
    resposta = json.loads(resposta)
    
    resposta['conversa'] = string
    resposta['session_id'] = session_id

    for key, value in resposta.items():
        resposta[key] = [value]

    df_conversas = pd.concat([df_conversas, pd.DataFrame(resposta)], axis = 0).reset_index(drop = True)

In [35]:
def mostra_infos_conversa(indice):

    display(df_conversas.loc[indice:indice,:])
    print("\n")
    print(df_conversas.loc[indice,"conversa"])
    print('\n')
    print(df_conversas.loc[indice,"summary"])
    print('\n')
    print(df_conversas.loc[indice,"improvement"])

## Conversa 1

In [36]:
mostra_infos_conversa(0)

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id
0,8,O cliente iniciou a conversa perguntando sobre...,O chatbot poderia ter perguntado se o cliente ...,False,0,None,Dia e hora da mensagem: 2024-11-27 21:27:48 \n...,10




Dia e hora da mensagem: 2024-11-27 21:27:48 
 User: Olá 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:07 
 Tempo de resposta: 0 days 00:00:19 
 Alzira: Olá, Roberto! 😊 Eu sou a Alzira, do Intense Motel. Como posso te ajudar hoje?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:19 
 User: Valor 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:37 
 Tempo de resposta: 0 days 00:00:18 
 Alzira: Claro! 😊 Para verificar os valores, você poderia me informar o dia e horário em que gostaria de se hospedar?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:49 
 User: Hoje a noite 

 -------------------------

### Observação

Reserva foi feita, porém, o Gregório não identificou que foi concluída. 

O cliente aparentemente pediu promoções do cardápio e a Alzira retornou promoções de quarto, sendo que o cliente já tinha reservado um quarto. O Gregório não identificou isso como um ponto de melhoria e também deu uma nota alta.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Conversa 2

In [37]:
mostra_infos_conversa(1)

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id
1,9,O cliente iniciou a conversa solicitando uma r...,O chatbot poderia ter oferecido um resumo das ...,True,158.20,2024-12-07T01:41:39,Dia e hora da mensagem: 2024-12-07 01:22:01 \n...,1




Dia e hora da mensagem: 2024-12-07 01:22:01 
 User: Oi 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-12-07 01:22:20 
 Tempo de resposta: 0 days 00:00:19 
 Alzira: Olá, Roberto! 😊 Eu sou a Alzira, do Intense Motel. Como posso te ajudar hoje?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-12-07 01:22:52 
 User: Reservar 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-12-07 01:23:10 
 Tempo de resposta: 0 days 00:00:18 
 Alzira: Ótimo, Roberto! Para que eu possa te ajudar com a reserva, qual dia e horário você gostaria de se hospedar? 🏨✨  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-12-07 01:36:17 
 User: Hoje 

 -----------------------------

### Observação

A nota e as informações de reserva forem bem captadas pelo Gregório. A data é discutível pois o cliente queria uma reserva imediata mas chegou bem próximo.
A sugestão de melhoria foi mediana.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Conversa 3

In [38]:
mostra_infos_conversa(4)

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id
4,4,O cliente iniciou a conversa solicitando uma r...,O chatbot poderia ter solicitado informações m...,False,0,None,Dia e hora da mensagem: 2025-01-07 19:19:35 \n...,28




Dia e hora da mensagem: 2025-01-07 19:19:35 
 User: Reserva 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:19:55 
 Tempo de resposta: 0 days 00:00:20 
 Alzira: ¡Hola, Roberto! 😊 Soy Alzira, del departamento de reservas del Intense Motel. ¿En qué día y horario te gustaría hospedarte?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:26:32 
 User: olá 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:26:50 
 Tempo de resposta: 0 days 00:00:18 
 Alzira: ¡Hola, Roberto! 😊 ¿En qué día y horario te gustaría hospedarte en el motel?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:45:28 
 Tempo de resposta: nan 
 Alzira:

### Observação

A nota até que foi coerente, pois aparentemente o cliente desistiu do atendimento, já que a Alzira começou a falar em espanhol (?). Entretanto, o Gregório não identificou esse erro e não sugeriu melhoria relacionado a isso.

# Melhorias

Agora, farei algumas modificações no prompt do Gregório para que ele capte as informações de maneira mais precisa e ofereça feedbacks e sugestões de melhoria mais coerentes e diversas.

Além disso, mantive as respostas anteriores para que o modelo possa evoluir ao longo do tempo, já que ajustei os parâmetros temperature, frequency_penalty e presence_penalty com o objetivo de ampliar seu repertório de ideias.

## Alteração no prompt

In [39]:
def analisar_conversa(texto_conversa, messages = None, temperature = 0.0, frequency_penalty = 0.0, presence_penalty = 0.0):

    system_prompt = (
        "Você é um analista de conversas especializado em reservas de motel. "
        "As conversas ocorrem entre um cliente (marcado como 'user') e uma chatbot chamada 'Alzira' (marcada como 'Alzira').  "
        "Cada mensagem traz data/hora, bem como o tempo que a chatbot levou para responder a última mensagem do cliente.  "
        "As mensagens estão separadas por linhas em branco e vários hifens.  "

        "Sua tarefa é analisar uma única conversa fornecida e retornar exclusivamente* um objeto JSON com a seguinte estrutura:"

        "{\n"
        '  "satisfaction": <número inteiro de 0 a 10>,\n'
        '  "summary": "<resumo breve e factual da conversa>",\n'
        '  "improvement": "<dicas/sugestões para melhorar a conversa>",\n'
        '  "reserva": "<true ou false>",\n'
        '  "income": "<valor numérico ou 0 caso não haja reserva>",\n'
        '  "date": "<data/hora da reserva em formato ISO 8601 ou null se não houver>"\n'
        "}\n\n"


        "Instrucoes:\n\n"
        "1. Satisfaction (0 a 10). Avalie quão satisfeito o cliente aparenta estar. Baseie-se especialmente em sinais de satisfação ou insatisfação no texto, no impacto de eventuais atrasos nas respostas e se o cliente parou de responder antes de reservar.\n\n"
        "2. Summary. Faca um resumo objetivo e conciso dos principais pontos da conversa: qual era a solicitacão do cliente, quais informações o chatbot forneceu, se houve confirmação de reserva etc. Não especifique o nome do cliente\n\n"
        "3. Improvement. Apresente sugestões para melhorar o atendimento da Alzira (clareza, cordialidade, detalhamento de opções, idioma conversado). Aqui você pode ter mais criatividade.\n\n"
        "4. Reserva (true ou false). Retorne true se houve uma confirmação clara de reserva (quarto escolhido, data agendada). Se houve a geração de link de pagamento, considere que ocorreu a reserva. Se o cliente apenas perguntou ou não concluiu, retorne false.\n\n"
        "5. Income (valor em reais). Se a conversa confirmou um quarto (e possivelmente adicionais), retorne o valor total. Se não houver reserva, retorne 0.\n\n"
        "6. Date. Se a data ou hora da reserva foi confirmada, retorne em formato ISO 8601 (por exemplo, 2025-02-01T14:00:00). Se não houver data agendada, retorne null.\n\n"

        "Muito importante: Retorne apenas o objeto JSON, sem texto adicional, cabeçalhos de cdigo ou comentários. Não invente informações que não estejam na conversa. Se algo não for mencionado, use defaults apropriados (reserva=false, income=0, date=null etc.). Seja coerente com o tom do cliente e os fatos descritos.\n"
    )


    if messages == None or len(messages) == 0:
        messages = [
            {"role": "developer", "content": system_prompt},
        ]


    user_prompt = (
        f"Aqui está a conversa:\n\n{texto_conversa}\n\n"
        "Lembre: Retorne exclusivamente JSON e nada mais. Sem códigos de linguagem natural, sem blocos de código, sem texto adicional. Somente o objeto JSON."
    )

    messages.append({"role": "user", "content": user_prompt})

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # ou outro modelo que você use
        messages=messages,
        temperature=temperature,
        presence_penalty = presence_penalty,
        frequency_penalty = frequency_penalty
    )

    #resposta = response.choices[0].message.content
    
    return response, messages

In [40]:
df_conversas = pd.DataFrame()
messages = []

for session_id in df.session_id.unique().tolist():

    df_aux = df.loc[df['session_id']==session_id].sort_values(by='created_at')

    string = ""

    for i, row in df_aux.iterrows():

        if row['remote'] == True:
            individuo = "User"
            string_aux = f"Dia e hora da mensagem: {row['created_at']} \n {individuo}: {row['content']} \n\n {100 * '-'} \n"
        
        elif row['remote'] == False:
            tempo_resposta = row['tempo_resposta']
            individuo = "Alzira"
            string_aux = f"Dia e hora da mensagem: {row['created_at']} \n Tempo de resposta: {tempo_resposta} \n {individuo}: {row['content']}  \n\n {100 * '-'} \n"

        string = string + string_aux

    response, messages = analisar_conversa(string, messages, 0.3, 0.5, 0.5)
    resposta = json.loads(response.choices[0].message.content)
    
    resposta['conversa'] = string
    resposta['session_id'] = session_id

    for key, value in resposta.items():
        resposta[key] = [value]

    df_conversas = pd.concat([df_conversas, pd.DataFrame(resposta)], axis = 0).reset_index(drop = True)


    messages.append({'role':'assistant', "content": response.choices[0].message.content})

In [41]:
df_conversas

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id
0,8,O cliente iniciou a conversa perguntando sobre...,A Alzira poderia ter dado mais informações sob...,True,121.1,2024-11-27T21:30:00,Dia e hora da mensagem: 2024-11-27 21:27:48 \n...,10
1,9,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter oferecido mais detalhes s...,True,158.2,2024-12-07T01:40:45,Dia e hora da mensagem: 2024-12-07 01:22:01 \n...,1
2,9,O cliente entrou em contato para saber sobre v...,A Alzira poderia ter oferecido mais informaçõe...,True,161.0,2025-01-10T00:00:00,Dia e hora da mensagem: 2025-01-07 12:10:47 \n...,5
3,7,O cliente entrou em contato para saber sobre a...,A Alzira poderia ter oferecido informações mai...,False,0.0,None,Dia e hora da mensagem: 2025-01-07 12:28:02 \n...,29
4,2,O cliente iniciou a conversa solicitando uma r...,A Alzira poderia ter incentivado o cliente a f...,False,0.0,None,Dia e hora da mensagem: 2025-01-07 19:19:35 \n...,28
5,9,O cliente entrou em contato para fazer uma res...,A Alzira poderia ter destacado as vantagens de...,True,349.6,2025-01-11T03:00:00,Dia e hora da mensagem: 2025-01-08 15:04:59 \n...,26
6,5,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter incentivado o cliente a t...,False,0.0,None,Dia e hora da mensagem: 2025-01-08 17:46:43 \n...,11
7,4,O cliente entrou em contato para saber os preç...,A Alzira poderia ter incentivado o cliente a t...,False,0.0,None,Dia e hora da mensagem: 2025-01-12 10:51:02 \n...,13
8,3,O cliente entrou em contato para perguntar sob...,A Alzira poderia ter tentado redirecionar a co...,False,0.0,None,Dia e hora da mensagem: 2025-01-17 00:20:38 \n...,14
9,8,O cliente entrou em contato para saber os valo...,A Alzira poderia ter incentivado o cliente a f...,False,0.0,None,Dia e hora da mensagem: 2025-01-22 16:42:16 \n...,15


## Conversa 1

In [42]:
mostra_infos_conversa(0)

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id
0,8,O cliente iniciou a conversa perguntando sobre...,A Alzira poderia ter dado mais informações sob...,True,121.1,2024-11-27T21:30:00,Dia e hora da mensagem: 2024-11-27 21:27:48 \n...,10




Dia e hora da mensagem: 2024-11-27 21:27:48 
 User: Olá 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:07 
 Tempo de resposta: 0 days 00:00:19 
 Alzira: Olá, Roberto! 😊 Eu sou a Alzira, do Intense Motel. Como posso te ajudar hoje?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:19 
 User: Valor 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:37 
 Tempo de resposta: 0 days 00:00:18 
 Alzira: Claro! 😊 Para verificar os valores, você poderia me informar o dia e horário em que gostaria de se hospedar?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2024-11-27 21:28:49 
 User: Hoje a noite 

 -------------------------

## Conversa 2

In [43]:
mostra_infos_conversa(4)

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id
4,2,O cliente iniciou a conversa solicitando uma r...,A Alzira poderia ter incentivado o cliente a f...,False,0.0,None,Dia e hora da mensagem: 2025-01-07 19:19:35 \n...,28




Dia e hora da mensagem: 2025-01-07 19:19:35 
 User: Reserva 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:19:55 
 Tempo de resposta: 0 days 00:00:20 
 Alzira: ¡Hola, Roberto! 😊 Soy Alzira, del departamento de reservas del Intense Motel. ¿En qué día y horario te gustaría hospedarte?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:26:32 
 User: olá 

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:26:50 
 Tempo de resposta: 0 days 00:00:18 
 Alzira: ¡Hola, Roberto! 😊 ¿En qué día y horario te gustaría hospedarte en el motel?  

 ---------------------------------------------------------------------------------------------------- 
Dia e hora da mensagem: 2025-01-07 19:45:28 
 Tempo de resposta: nan 
 Alzira:

## Observações


Houve uma melhoria na capacidade do Gregório de identificar quando uma reserva foi realmente efetuada.

As sugestões de melhoria tornaram-se mais criativas e diversificadas, trazendo novas ideias. No entanto, ainda há desafios na captação de certos detalhes, como o atendimento em espanhol e a identificação correta de quando a Alzira ofereceu promoções de quartos em vez de promoções do cardápio, conforme solicitado pelo cliente.

Outro ponto a ser otimizado é a redução do consumo de tokens, a fim de minimizar os custos com a API.

Portanto, ainda existem oportunidades de aprimoramento para que o Gregório seja mais assertivo na extração de informações, gere sugestões mais eficazes e, ao mesmo tempo, mantenha a eficiência no uso da API.

# Salvar no banco de dados

## Criação de colunas

In [44]:
df_conversas['id'] = np.arange(1, df_conversas.shape[0] + 1)

In [45]:
df_aux = df.groupby(['session_id'])[['created_at']].min().reset_index()
df_aux

,session_id,created_at
0,1,2024-12-07 01:22:01
1,5,2025-01-07 12:10:47
2,10,2024-11-27 21:27:48
3,11,2025-01-08 17:46:43
4,13,2025-01-12 10:51:02
5,14,2025-01-17 00:20:38
6,15,2025-01-22 16:42:16
7,16,2025-01-27 14:12:34
8,17,2025-01-28 14:55:35
9,18,2025-01-29 18:28:38


In [46]:
df_conversas = df_conversas.merge(df_aux, on = 'session_id', how = 'left')
df_conversas

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id,id,created_at
0,8,O cliente iniciou a conversa perguntando sobre...,A Alzira poderia ter dado mais informações sob...,True,121.1,2024-11-27T21:30:00,Dia e hora da mensagem: 2024-11-27 21:27:48 \n...,10,1,2024-11-27 21:27:48
1,9,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter oferecido mais detalhes s...,True,158.2,2024-12-07T01:40:45,Dia e hora da mensagem: 2024-12-07 01:22:01 \n...,1,2,2024-12-07 01:22:01
2,9,O cliente entrou em contato para saber sobre v...,A Alzira poderia ter oferecido mais informaçõe...,True,161.0,2025-01-10T00:00:00,Dia e hora da mensagem: 2025-01-07 12:10:47 \n...,5,3,2025-01-07 12:10:47
3,7,O cliente entrou em contato para saber sobre a...,A Alzira poderia ter oferecido informações mai...,False,0.0,None,Dia e hora da mensagem: 2025-01-07 12:28:02 \n...,29,4,2025-01-07 12:28:02
4,2,O cliente iniciou a conversa solicitando uma r...,A Alzira poderia ter incentivado o cliente a f...,False,0.0,None,Dia e hora da mensagem: 2025-01-07 19:19:35 \n...,28,5,2025-01-07 19:19:35
5,9,O cliente entrou em contato para fazer uma res...,A Alzira poderia ter destacado as vantagens de...,True,349.6,2025-01-11T03:00:00,Dia e hora da mensagem: 2025-01-08 15:04:59 \n...,26,6,2025-01-08 15:04:59
6,5,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter incentivado o cliente a t...,False,0.0,None,Dia e hora da mensagem: 2025-01-08 17:46:43 \n...,11,7,2025-01-08 17:46:43
7,4,O cliente entrou em contato para saber os preç...,A Alzira poderia ter incentivado o cliente a t...,False,0.0,None,Dia e hora da mensagem: 2025-01-12 10:51:02 \n...,13,8,2025-01-12 10:51:02
8,3,O cliente entrou em contato para perguntar sob...,A Alzira poderia ter tentado redirecionar a co...,False,0.0,None,Dia e hora da mensagem: 2025-01-17 00:20:38 \n...,14,9,2025-01-17 00:20:38
9,8,O cliente entrou em contato para saber os valo...,A Alzira poderia ter incentivado o cliente a f...,False,0.0,None,Dia e hora da mensagem: 2025-01-22 16:42:16 \n...,15,10,2025-01-22 16:42:16


## Processamento

In [47]:
df_conversas['id'] = df_conversas['id'].astype(int)
df_conversas['satisfaction'] = df_conversas['satisfaction'].astype(int)
df_conversas['summary'] = df_conversas['summary'].astype(str)
df_conversas['improvement'] = df_conversas['improvement'].astype(str)
df_conversas['reserva'] = df_conversas['reserva'].astype(int)
df_conversas['income'] = df_conversas['income'].astype(float)
df_conversas['date'] = pd.to_datetime(df_conversas['date'])
df_conversas['conversa'] = df_conversas['conversa'].astype(str)
df_conversas['session_id'] = df_conversas['session_id'].astype(int)

In [48]:
df_conversas

,satisfaction,summary,improvement,reserva,income,date,conversa,session_id,id,created_at
0,8,O cliente iniciou a conversa perguntando sobre...,A Alzira poderia ter dado mais informações sob...,1,121.1,2024-11-27 21:30:00,Dia e hora da mensagem: 2024-11-27 21:27:48 \n...,10,1,2024-11-27 21:27:48
1,9,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter oferecido mais detalhes s...,1,158.2,2024-12-07 01:40:45,Dia e hora da mensagem: 2024-12-07 01:22:01 \n...,1,2,2024-12-07 01:22:01
2,9,O cliente entrou em contato para saber sobre v...,A Alzira poderia ter oferecido mais informaçõe...,1,161.0,2025-01-10 00:00:00,Dia e hora da mensagem: 2025-01-07 12:10:47 \n...,5,3,2025-01-07 12:10:47
3,7,O cliente entrou em contato para saber sobre a...,A Alzira poderia ter oferecido informações mai...,0,0.0,NaT,Dia e hora da mensagem: 2025-01-07 12:28:02 \n...,29,4,2025-01-07 12:28:02
4,2,O cliente iniciou a conversa solicitando uma r...,A Alzira poderia ter incentivado o cliente a f...,0,0.0,NaT,Dia e hora da mensagem: 2025-01-07 19:19:35 \n...,28,5,2025-01-07 19:19:35
5,9,O cliente entrou em contato para fazer uma res...,A Alzira poderia ter destacado as vantagens de...,1,349.6,2025-01-11 03:00:00,Dia e hora da mensagem: 2025-01-08 15:04:59 \n...,26,6,2025-01-08 15:04:59
6,5,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter incentivado o cliente a t...,0,0.0,NaT,Dia e hora da mensagem: 2025-01-08 17:46:43 \n...,11,7,2025-01-08 17:46:43
7,4,O cliente entrou em contato para saber os preç...,A Alzira poderia ter incentivado o cliente a t...,0,0.0,NaT,Dia e hora da mensagem: 2025-01-12 10:51:02 \n...,13,8,2025-01-12 10:51:02
8,3,O cliente entrou em contato para perguntar sob...,A Alzira poderia ter tentado redirecionar a co...,0,0.0,NaT,Dia e hora da mensagem: 2025-01-17 00:20:38 \n...,14,9,2025-01-17 00:20:38
9,8,O cliente entrou em contato para saber os valo...,A Alzira poderia ter incentivado o cliente a f...,0,0.0,NaT,Dia e hora da mensagem: 2025-01-22 16:42:16 \n...,15,10,2025-01-22 16:42:16


## Salvar

In [49]:
df_analise = df_conversas[['session_id', 'satisfaction', 'summary', 'improvement', 'created_at', 'reserva','income', 'date']].rename(columns = {'date':"data_reserva"})
df_analise

,session_id,satisfaction,summary,improvement,created_at,reserva,income,data_reserva
0,10,8,O cliente iniciou a conversa perguntando sobre...,A Alzira poderia ter dado mais informações sob...,2024-11-27 21:27:48,1,121.1,2024-11-27 21:30:00
1,1,9,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter oferecido mais detalhes s...,2024-12-07 01:22:01,1,158.2,2024-12-07 01:40:45
2,5,9,O cliente entrou em contato para saber sobre v...,A Alzira poderia ter oferecido mais informaçõe...,2025-01-07 12:10:47,1,161.0,2025-01-10 00:00:00
3,29,7,O cliente entrou em contato para saber sobre a...,A Alzira poderia ter oferecido informações mai...,2025-01-07 12:28:02,0,0.0,NaT
4,28,2,O cliente iniciou a conversa solicitando uma r...,A Alzira poderia ter incentivado o cliente a f...,2025-01-07 19:19:35,0,0.0,NaT
5,26,9,O cliente entrou em contato para fazer uma res...,A Alzira poderia ter destacado as vantagens de...,2025-01-08 15:04:59,1,349.6,2025-01-11 03:00:00
6,11,5,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter incentivado o cliente a t...,2025-01-08 17:46:43,0,0.0,NaT
7,13,4,O cliente entrou em contato para saber os preç...,A Alzira poderia ter incentivado o cliente a t...,2025-01-12 10:51:02,0,0.0,NaT
8,14,3,O cliente entrou em contato para perguntar sob...,A Alzira poderia ter tentado redirecionar a co...,2025-01-17 00:20:38,0,0.0,NaT
9,15,8,O cliente entrou em contato para saber os valo...,A Alzira poderia ter incentivado o cliente a f...,2025-01-22 16:42:16,0,0.0,NaT


In [50]:
df_analise['created_at'] = df_analise['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

## Adicionar session faltantes

In [51]:
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)
cur = conn.cursor()

In [52]:
session_ids_analysis = df_analise['session_id'].unique()
cur.execute("SELECT id FROM session;")
session_ids_in_session = [row[0] for row in cur.fetchall()]

missing_session_ids = set(session_ids_analysis) - set(session_ids_in_session)

df_aux = df.loc[df['session_id'].isin(missing_session_ids)].sort_values(by='created_at').drop_duplicates(subset=['session_id'],keep='first')[['session_id','motel_id','created_at']]
df_aux

,session_id,motel_id,created_at


In [53]:
# Inserir os dados faltantes no banco
for _, row in df_aux.iterrows():
    cur.execute(
        """
        INSERT INTO session (id, motel_id, created_at)
        VALUES (%s, %s, %s)
        """,
        (row['session_id'], row['motel_id'], row['created_at'])
    )

# Confirmar mudanças no banco
conn.commit()

In [54]:
df_analise

,session_id,satisfaction,summary,improvement,created_at,reserva,income,data_reserva
0,10,8,O cliente iniciou a conversa perguntando sobre...,A Alzira poderia ter dado mais informações sob...,2024-11-27 21:27:48,1,121.1,2024-11-27 21:30:00
1,1,9,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter oferecido mais detalhes s...,2024-12-07 01:22:01,1,158.2,2024-12-07 01:40:45
2,5,9,O cliente entrou em contato para saber sobre v...,A Alzira poderia ter oferecido mais informaçõe...,2025-01-07 12:10:47,1,161.0,2025-01-10 00:00:00
3,29,7,O cliente entrou em contato para saber sobre a...,A Alzira poderia ter oferecido informações mai...,2025-01-07 12:28:02,0,0.0,NaT
4,28,2,O cliente iniciou a conversa solicitando uma r...,A Alzira poderia ter incentivado o cliente a f...,2025-01-07 19:19:35,0,0.0,NaT
5,26,9,O cliente entrou em contato para fazer uma res...,A Alzira poderia ter destacado as vantagens de...,2025-01-08 15:04:59,1,349.6,2025-01-11 03:00:00
6,11,5,O cliente entrou em contato para reservar uma ...,A Alzira poderia ter incentivado o cliente a t...,2025-01-08 17:46:43,0,0.0,NaT
7,13,4,O cliente entrou em contato para saber os preç...,A Alzira poderia ter incentivado o cliente a t...,2025-01-12 10:51:02,0,0.0,NaT
8,14,3,O cliente entrou em contato para perguntar sob...,A Alzira poderia ter tentado redirecionar a co...,2025-01-17 00:20:38,0,0.0,NaT
9,15,8,O cliente entrou em contato para saber os valo...,A Alzira poderia ter incentivado o cliente a f...,2025-01-22 16:42:16,0,0.0,NaT


In [55]:
# Query de inserção
sql_insert = """
    INSERT INTO analysis (session_id, satisfaction, summary, improvement, created_at)
    VALUES (%s, %s, %s, %s, %s)
"""

# Converter DataFrame para lista de tuplas e executar inserção
data_to_insert = df_analise[["session_id", "satisfaction", "summary", "improvement", "created_at"]].to_records(index=False).tolist()
cur.executemany(sql_insert, data_to_insert)

# Confirmar mudanças no banco
conn.commit()

In [56]:
cur.close()
conn.close()

## Observações

Não consegui adicionar novas colunas na tabela analysis no banco de dados. Esse é um ponto que precisa ser aprimorado, pois a inclusão dessas colunas permitiria armazenar mais informações sobre as conversas, possibilitando análises mais detalhadas e uma maior riqueza de dados para insights futuros.

Além disso, o arquivo .py não está otimizado, pois pode ser modularizado de uma forma bem melhor.